# Machine Learning pour prédire les résultats de contrôles sanitaires

Dans ce notebook, vous trouvez notre code d'entrainement et validation automatique d'algorithmes de Machine Learning pour prédire les résultats de contrôles sanitaires à partir du jeu de données *Résultats des contrôles officiels sanitaires : dispositif d’information « Alim’confiance »*.

**dataset:**https://www.data.gouv.fr/fr/datasets/resultats-des-controles-officiels-sanitaires-dispositif-dinformation-alimconfiance/ <br>
**tâche:** Classification (Classifcation multi-classe)<br>
**target variable:** *Synthese_eval_sanit*

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import io
import requests
#pour l'entraînement automatique:
from supervised.automl import AutoML
#pour générer un rapport html des résultats:
import IPython
import markdown

## 1. Importation des données

On importe le dataset à partir de son url.

In [4]:
url_cont_san = "https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72" #données brutes en csv
r = requests.get(url_cont_san,allow_redirects=True)
df=pd.read_csv(io.StringIO(r.content.decode('utf-8')), sep=';')


In [ ]:
df=pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72",sep=";",encoding='utf-8')

## 2. Preprocessing et feature engineering

Le code de cette partie à été partiellement pris du notebook fait par Julien Denes en Introduction au ML: https://colab.research.google.com/drive/1OgqIFNCkEi4YL9Xr7R0yegWbhZGG0HiV#scrollTo=YkR83PK6V_Nr

In [5]:
#dans la colonne 'filtre', plusieurs filtres sont parfois donnés.
#on en séléctionne un seulement et on gère ses valeurs manquantes:
df['filtre'] = df['filtre'].str.split('\|').str[0]
df['filtre'] = df['filtre'].fillna('NA')

#modification du type des variables latitude et longitude:
df['latitude'] = df['geores'].str.split(',').str[0].astype(float)
df['longitude'] = df['geores'].str.split(',').str[1].astype(float)

#modification du type de la variable réponse
df['Synthese_eval_sanit'] = df['Synthese_eval_sanit'].astype('category').cat.codes

#modification du type de la variable 'date d'inspection
df['Date_inspection'] = pd.to_datetime(df['Date_inspection'], format='%Y-%m-%dT%H:%M:%S', utc=True)

In [6]:
#création de plusieurs nouvelles variables:
df['has_agrement'] = pd.notnull(df['Agrement']).astype(int)

#variable département:
df['dept'] = [int(r[:2]) for r in df['Code_postal']]

#variables temporelles
df['year'] = df['Date_inspection'].dt.year
df['month'] = df['Date_inspection'].dt.month
df['weekday'] = df['Date_inspection'].dt.weekday

#comptage et score des controles 
df['count_controls_dept'] = df.groupby('dept')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_dept'] = df.groupby('dept')['Synthese_eval_sanit'].transform(lambda x: x.mean())
df['count_controls_filtre'] = df.groupby('filtre')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_filtre'] = df.groupby('filtre')['Synthese_eval_sanit'].transform(lambda x: x.mean())
df['ods_type_activite'] = df['ods_type_activite'].fillna('NA')
df['count_controls_activite'] = df.groupby('ods_type_activite')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_activite'] = df.groupby('ods_type_activite')['Synthese_eval_sanit'].transform(lambda x: x.mean())
df['count_controls_wday'] = df.groupby('weekday')['Synthese_eval_sanit'].transform(lambda x: x.count())
df['score_controls_wday'] = df.groupby('weekday')['Synthese_eval_sanit'].transform(lambda x: x.mean())

In [7]:
#élimination des colonnes inutiles
drop_cols = ['APP_Libelle_etablissement', 'Code_postal', 'SIRET', 'Libelle_commune',
             'APP_Libelle_activite_etablissement', 'Numero_inspection', 'Date_inspection',
             'Agrement', 'geores', 'ods_type_activite','Adresse_2_UA']
df = df.drop(drop_cols, axis=1)

In [8]:
#transformation des variables catégorielles
categorical_features = ['dept', 'filtre', 'month', 'weekday']

for c in categorical_features:
    df[c] = df[c].astype('category').cat.codes

## 3. Train/test splitting

On n'effectue pas de séléction de variables. On définit la variable réponse et les variables explicatives.

In [9]:
y = df['Synthese_eval_sanit'].values
X = df.drop(columns = ['Synthese_eval_sanit'])

On partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 4. AutoML

On entraîne, test et valide automatiquement plusieurs algorithmes de ML sur le dataset avec AutoML (plus d'infos ici: https://supervised.mljar.com/) et on génère un html des métriques et des paramètres du modèle.

In [ ]:
automl = AutoML(total_time_limit=5*60,mode='Explain',random_state=42)
#fit model
automl.fit(X_train,y_train)
#predictions
predictions = automl.predict(X_test)
#generate html report
automl.report()

### Preprocessing fait par AutoML

Il n'est pas évident de comprendre, à partir du rapport html qui est généré, quelles tâches de preprocessing sont faites dans la **pipeline de AutoML** qui permet d' aboutir aux résultats observés.  <br>
Pour mieux comprendre les résultats, on peut s'intéresser au fichier `framework.json` de chaque modèle. Par exemple, on peut consulter ce fichier pour le modèle `Xgboost` ici:
https://github.com/etalab-ia/DGML/blob/main/docs/automodels/c763b24a-a0fe-4e77-9586-3d5453c631cd/4_Default_Xgboost/framework.json<br>
On observe:
1. L'imputation des **valeurs manquantes** est faite en remplaçant par la valeur médiane
2. Un **encoding** des variables catégorielles est fait en transformant le nom de chaque catégorie en un entier
 
Ces informations sont disponibles pour tous les algorithmes.